<a href="https://colab.research.google.com/github/bobby1/UCSC-Natural-Language-Processing/blob/main/AISV_801_lab2_Wen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AISV.801.(6) Natural Language Processing

Professor Joseph Meyer

Bobby Wen

Assignment 2

      1. Download imdb dataset from hugging face (the train split)
      2. Create several functions:
      a. Exploratory function
            i.Prints out
                1. Top n most common words
                2. Average text length
                3. Longest text length
        b. Sentence tokenization function
            i. Apply sentence tokenization to the text
            ii. Return a dataframe that expands each text into multiple rows for each sentence (e.g., if the first text has five sentences, there are now five rows for the original text)
        c. Text cleaning function
            i. Remove non-alphanumeric characters
            ii. Remove excess whitespace (extra spaces)
            iii. Remove stop words
            iv. Lemmatizes text
            v. Creates a new column in the dataframe: clean_text
        c. Vectorization function
            i. Returns two objects:
                1. TF-IDF weighted document term matrix
                2. Bigram document term matrix


***Pre-requisits***

**Datasets** is a fast and efficient library to easily share and load datasets.

Build with interoperability with PyTorch, Tensorflow 2, Pandas and Numpy
Lighweight and fast library with a transparent and pythonic API.  It
thrives on large datasets: frees you from RAM memory limits, all datasets are memory-mapped on drive by default.

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00


**Transformers** also known as pytorch-transformers and pytorch-pretrained-bert provides general-purpose architectures like XLM, DistilBert, XLNet, BERT, GPT-2, RoBERTa, etc for Natural Language Understanding(NLU) and Natural Language Generation (NLG) with over 30+ pre-trained models in 100+ languages and deep interoperability between Jax, PyTorch, and TensorFlow.

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 132.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.5 MB/s eta 0:00:00


The **Natural Language Toolkit (NLTK)** is a platform used for building Python programs that work with human language data for applying in statistical natural language processing (NLP). It contains text processing libraries for tokenization, parsing, classification, stemming, tagging and semantic reasoning.



In [ ]:
!pip install nltk

**IMDB** data set is a Large Movie Review Dataset. This is a dataset for binary sentiment classification. It contains a set of 25,000 highly popular movie reviews for training, and 25,000 for testing.

In [ ]:
#Load imdb dataset
from datasets import load_dataset

dataset = load_dataset('imdb', split='train')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


**Pandas** is an open source library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

In [ ]:
# load dataset to pre-process
import pandas as pd

reviews = dataset["text"]

In [ ]:
all_reviews = " ".join(reviews)

**AutoTokenizer** are tokenizers responsible for preprocessing text into an array of numbers as inputs to a model. Multiple rules govern the tokenization process, including how to split a word and at what level words should be split.

**DistilBERT**, a distilled version of BERT. DistilBERT is a small, fast, cheap and light Transformer model based on the BERT architecture. Knowledge distillation is performed during the pre-training phase to reduce the size of a BERT model by 40%

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
words = tokenizer.tokenize(all_reviews)

Token indices sequence length is longer than the specified maximum sequence length for this model (7796783 > 512). Running this sequence through the model will result in indexing errors


In natural language processing, useless words (data), are referred to as stop words.  **Stop word** are commonly used word (such as “the”, “a”, “an”, “in”) that are ignored.

**Lemmatization** is the process of reducing a word to its base or dictionary form, known as the lemma. For example, the lemma of the word “cats” is “cat”, and the lemma of “running” is “run”.
The NLTK library used for natural language processing has a modules, WordNet Lemmatizer, which can be used to perform lemmatization on words.

The Python "**re**" module provides regular expression support. The re.search() method takes a regular expression pattern and a string and searches for that pattern within the string.

Python **Counter** is an unordered collection where elements are stored as Dict keys and their count as dict values. Counter elements count can be positive, zero, or negative integers. There is no restriction on its keys and values. The values are intended to be numbers, but we can store other objects too.

In [ ]:
# a. Exploratory function
#    i.Prints out
#        1. Top n most common words
#        2. Average text length
#        3. Longest text length
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from collections import Counter

def explore_dataset(n=10):
    # Load the IMDb dataset
    dataset = load_dataset("imdb")
    reviews = dataset["train"]["text"]

    # Top n most common words
    words = " ".join(reviews).split()
    word_counts = Counter(words)
    top_words = word_counts.most_common(n)

    print(f"Top {n} most common words:")
    for word, count in top_words:
        print(f"{word}: {count}")

    # Average text length
    total_length = sum(len(review) for review in reviews)
    average_length = total_length / len(reviews)
    print(f"\nAverage text length: {average_length:.0f} characters")

    # Longest text length
    longest_text = max(reviews, key=len)
    longest_length = len(longest_text)
    print(f"Longest text length: {longest_length} characters")
    print (f"Longest text", longest_text)

In [ ]:
# Run the explore data set
explore_dataset(n=15)

  0%|          | 0/3 [00:00<?, ?it/s]

Top 15 most common words:
the: 287032
a: 155096
and: 152664
of: 142972
to: 132568
is: 103229
in: 85580
I: 65973
that: 64560
this: 57199
it: 54439
/><br: 50935
was: 46698
as: 42510
with: 41721

Average text length: 1325 characters
Longest text length: 13704 characters
Longest text Match 1: Tag Team Table Match Bubba Ray and Spike Dudley vs Eddie Guerrero and Chris Benoit Bubba Ray and Spike Dudley started things off with a Tag Team Table Match against Eddie Guerrero and Chris Benoit. According to the rules of the match, both opponents have to go through tables in order to get the win. Benoit and Guerrero heated up early on by taking turns hammering first Spike and then Bubba Ray. A German suplex by Benoit to Bubba took the wind out of the Dudley brother. Spike tried to help his brother, but the referee restrained him while Benoit and Guerrero ganged up on him in the corner. With Benoit stomping away on Bubba, Guerrero set up a table outside. Spike dashed into the ring and somersaulted o

The **Punkt** sentenceTokenizer is a NLTK module. PUNKT is an unsupervised trainable model which can be trained on unlabeled data.

In [ ]:
#    b. Sentence tokenization function
#        i. Apply sentence tokenization to the text
#        ii. Return a dataframe that expands each text into multiple rows for each sentence (e.g., if the first text has five sentences, there are now five rows for the original text)
nltk.download('punkt')

def tokenize_sentence(dataset_name):
    # Load the dataset
    dataset = load_dataset(dataset_name)
    reviews = dataset["train"]["text"]

    # Sentence tokenization and expansion
    expanded_rows = []
    for idx, review in enumerate(reviews):
        sentences = sent_tokenize(review)
        for sentence in sentences:
            expanded_rows.append({"Text": review, "Sentence": sentence, "Index": idx})

    # Create a dataframe
    df = pd.DataFrame(expanded_rows)

    return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = tokenize_sentence("imdb")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
### DEBUG code
df.head()

,Text,Sentence,Index
0,I rented I AM CURIOUS-YELLOW from my video sto...,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,I rented I AM CURIOUS-YELLOW from my video sto...,I also heard that at first it was seized by U....,0
2,I rented I AM CURIOUS-YELLOW from my video sto...,In particular she wants to focus her attention...,0
3,I rented I AM CURIOUS-YELLOW from my video sto...,In between asking politicians and ordinary den...,0
4,I rented I AM CURIOUS-YELLOW from my video sto...,"Really, the sex and nudity scenes are few and ...",0


**WordNet** is a lexical database of English. Using synsets, helps find conceptual relationships between words such as hypernyms, hyponyms, synonyms, antonyms etc.

Open Multilingual Wordnet (**OMW**) is to make it easy to use wordnets in multiple languages. It extracted and normalized the data and linkes it to Princeton WordNet 3.0

In [ ]:
# c. Text cleaning function
#      i. Remove non-alphanumeric characters
#      ii. Remove excess whitespace (extra spaces)
#      iii. Remove stop words
#      iv. Lemmatizes text
#      v. Creates a new column in the dataframe: clean_text
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

def clean_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r"[^\w\s]", "", text)

    # Remove excess whitespace
    text = re.sub(r"\s+", " ", text)

    # Tokenize the text into words
    words = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word.lower() not in stop_words]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join the words back into a clean text
    clean_text = " ".join(words)

    return clean_text

def clean_and_add_column(dataset_name, column_name):
    # Load the dataset
    dataset = load_dataset(dataset_name)
    df = pd.DataFrame(dataset["train"])

    # Apply cleaning function to the specified column
    df["clean_text"] = df[column_name].apply(clean_text)

    return df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
df = clean_and_add_column("imdb", "text")

  0%|          | 0/3 [00:00<?, ?it/s]

**Tf–idf** (also TF*IDF, TFIDF, TF–IDF, or Tf–idf), short for term frequency–inverse document frequency, is a numerical statistic that reflects how important a word is to a document in a collection or corpus. It is used as a weighting factor in information retrieval, text mining, and user modeling searches.

**Bigrams** or digrams is a sequence of two adjacent elements from a string of tokens, which are typically letters, syllables, or words. A bigram is an n-gram for n=2. The frequency distribution of every bigram in a string is commonly used for simple statistical analysis of text in many applications, including in computational linguistics, cryptography, and speech recognition.

In [ ]:
#c. Vectorization function
#    i. Returns two objects:
#        1. TF-IDF weighted document term matrix
#        2. Bigram document term matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

def vectorization(dataset_name, column_name):
    # Load the dataset
    dataset = load_dataset(dataset_name)
    df = dataset["train"]

    # Get the text data
    text_data = df[column_name]

    # Create TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)

    # Create bigram vectorizer
    bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))
    bigram_matrix = bigram_vectorizer.fit_transform(text_data)

    return tfidf_matrix, bigram_matrix

In [ ]:
tfidf_matrix, bigram_matrix = vectorization("imdb", "text")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(tfidf_matrix, bigram_matrix)

  (0, 44241)	0.02741580670031791
  (0, 30118)	0.019001856191236614
  (0, 19423)	0.03552045878153692
  (0, 9881)	0.016125905071342508
  (0, 34005)	0.06326278825770798
  (0, 52530)	0.08211215460952387
  (0, 45805)	0.025635180313682976
  (0, 51062)	0.10384421522588236
  (0, 41855)	0.07729842262309586
  (0, 63781)	0.0687354802088816
  (0, 72093)	0.06639577209664187
  (0, 3715)	0.040895729601112804
  (0, 3017)	0.057027658871043156
  (0, 66344)	0.06942571291105844
  (0, 6334)	0.019040391912466495
  (0, 43572)	0.04296547021414409
  (0, 40421)	0.02916884679234349
  (0, 49147)	0.028996106751842388
  (0, 59724)	0.06397347066316686
  (0, 35787)	0.02258783502837706
  (0, 66299)	0.02735947650253524
  (0, 53602)	0.04125986122660155
  (0, 52662)	0.08135856482450572
  (0, 4430)	0.0657332787546271
  (0, 24536)	0.057631647085163364
  :	:
  (24999, 61682)	0.1098380688043193
  (24999, 59025)	0.08300218364271954
  (24999, 28545)	0.05698643001915573
  (24999, 26333)	0.08293484796071239
  (24999, 30211)	0.04